In [1]:
import numpy as np
import os

np.random.seed(41)

In [20]:
def sample_inits(N):
    """
    Samples initial values of timeseries from state space of latent factors.

    Args:
        N:  Number of samples to take. (N_max = 737280)
    """
    inits = np.zeros([N, 6], dtype=int)
    # Sample from entire latent space
    N_max = 737280
    rand_idxs = np.random.choice(N_max, N, replace=False)
    inits_idxs = np.unravel_index(rand_idxs, (1,3,6,40,32,32))

    # Inits from all combinations, except y. Randomly sample y.
#     all_but_y = np.array(np.meshgrid(np.arange(1), 
#                                      np.arange(3), 
#                                      np.arange(6), 
#                                      np.arange(40), 
#                                      np.arange(32))).T.reshape(-1,5)
#     all_but_y_stack = np.concatenate((all_but_y, all_but_y, all_but_y), axis=0)
#     y_sample = np.random.choice(np.arange(1), all_but_y_stack.shape[0])
#     inits_test = np.hstack((all_but_y_stack, np.array([y_sample]).T))

    # Choose inits from first N in latent space. Uncomment to apply.
#     inits_idxs = np.unravel_index(range(N), (1,3,6,40,32,32))

    for i in range(6):
        inits[:,i] = inits_idxs[i]

    # Additionally sample scale and shape. Uncomment to apply.
    # shapes = np.random.choice((0,1,2), N)
    # scales = np.random.choice((0,1,2,3,4,5), N)
    # inits[:,1] = shapes
    # inits[:,2] = scales

    # Hardcoded: remove shape, scale variation. Uncomment to apply.
    # inits[:,0:3] = 0

    return inits

In [3]:
def create_factors(inits, periods, length):
    """
    Creates latent factor tensor to sample from.

    Args:
        inits:    [M] initial values of latent factors
        periods:  [M] periods of latent factors
        length: [N] length of resulting tensor
    Output:
        factors:[NxM] tensor of latent factors
    """

    factors = np.zeros([length, periods.size], dtype=int)
    amplitudes = [0, 2, 5, 39, 31, 31] # Hardcoded for DSprites for now
    xaxis = np.arange(0,length,1)

    for i in range(0,periods.size):
        if amplitudes[i]:
            c = np.arccos(1 - 2*inits[i]/amplitudes[i])
        else:
            c = 0
        factors[:,i] = np.rint(-0.5*amplitudes[i] * np.cos(periods[i] * xaxis * 2*np.pi/length + c) + 0.5*amplitudes[i])

    return factors

In [4]:
def create_input(N, periods, length):
    """
    Generates N dimensional input for GP-VAE model.

    Args:
        N:  Dimensionality of input
        periods:  [M] periods of latent factors
        length: [N] length of resulting tensor
    """
#     dsp = dsprites.DSprites()
    # Hardcoded for now
    random_seed = 42
    random_state = np.random.RandomState(random_seed)

    inits = sample_inits(N)
    print('Inits shape: {}'.format(inits.shape))

    input = np.zeros([N, length, 64*64])

    all_factors = np.empty([N,6,length])

    for i in range(N):
        factors = create_factors(inits[i,:], periods, length)
        # print('FACTORS SHAPE {}'.format(factors.shape))
        all_factors[i,:,:] = factors.transpose()
#         dataset = np.squeeze(dsp.sample_observations_from_factors_no_color(factors=factors, random_state=random_state))
        # print('DATASET SHAPE {}'.format(dataset.shape))
#         dataset = dataset.reshape(dataset.shape[0], 64*64)
        # print('DATASET RESHAPE SHAPE {}'.format(dataset.shape))
#         input[i,:,:] = dataset

    print(all_factors.shape)

    return input.astype('float32'), all_factors

In [21]:
periods = np.array([0, 0, 0, 0.5, 1, 2]) # Should be integer multiples of 0.5
length = 10

_, all_factors = create_input(69120, periods, length)

Inits shape: (69120, 6)
(69120, 6, 10)


In [22]:
shape_all = all_factors.shape
f_all = np.reshape(np.transpose(all_factors, (0,2,1)), (shape_all[0]*shape_all[2], shape_all[1]))
f_all_flat = np.ravel_multi_index(np.transpose(f_all.astype(int)), (1,3,6,40,32,32), order='F')

In [23]:
shape_all

(69120, 6, 10)

In [24]:
f_all_unique = np.unique(f_all_flat)
print(f_all_unique.shape)

(310060,)


In [43]:
test_grid = np.array(np.meshgrid(np.arange(1), np.arange(3), np.arange(6), np.arange(40), np.arange(32))).T.reshape(-1,5)

In [71]:
test_grid.shape

(23040, 5)

In [122]:
inits = sample_inits(23040)
print(inits.shape)

(69120, 6)


In [2]:
data_path = '/cluster/work/grlab/projects/projects2020_disentangled_gpvae/data/dsprites/factors_100k_5k.npz'

data_full = np.load(data_path)
factors_train = data_full['factors_train']
factors_test = data_full['factors_test']

In [3]:
factors_test.shape

(5000, 6, 10)

In [4]:
# z_reshape = np.reshape(np.transpose(z, (0,2,1)),(z_shape[0]*z_shape[2],z_shape[1]))
test_shape = factors_test.shape
train_shape = factors_train.shape
f_test = np.reshape(np.transpose(factors_test, (0,2,1)), (test_shape[0]*test_shape[2], test_shape[1]))
f_train = np.reshape(np.transpose(factors_train, (0,2,1)), (train_shape[0]*train_shape[2], train_shape[1]))

f_test_flat = np.ravel_multi_index(np.transpose(f_test.astype(int)), (1,3,6,40,32,32), order='F')
f_train_flat = np.ravel_multi_index(np.transpose(f_train.astype(int)), (1,3,6,40,32,32), order='F')

In [5]:
f_test_unique = np.unique(f_test_flat)
f_train_unique = np.unique(f_train_flat)

In [7]:
f_train_unique.shape

(374662,)

In [22]:
np.ravel_multi_index(np.transpose(test), (1,3,6,40,32,32), order='F')

array([1, 2])